In [32]:
import os

import pandas as pd

golds = pd.read_excel('sql_golds.xlsx')
valid_data = golds[["question", "answer"]].rename(columns={"answer": "gold_answers", "question": "questions"})

Оффлайн оценка на собранных данных. Мы все же назначли людей покопаться в них.

In [5]:
from gradio_client import Client

class Qwen:
    def __init__(self, system_prompt: str):
        self.client = Client("Qwen/Qwen2.5")
        self.system_prompt = system_prompt

    def invoke(self, input_text: str):
        result = self.client.predict(
            query=input_text,
            history=[],
            system=self.system_prompt,
            radio="72B",
            api_name="/model_chat"
        )[1][0][-1]['text']
        return result


assistant = Qwen("You are helpful finance assistant. You know some information about user: his average income is 100 000 rubs na dhe loves pony")

Loaded as API: https://qwen-qwen2-5.hf.space ✔


In [33]:
from typing import Iterable
from tqdm import tqdm
from pathlib import Path

def get_answers(model, questions: Iterable[str]):
    return [model.invoke(q) for q in tqdm(questions, desc="Making predictions")]
        
valid_base_path = Path("valid_set_base.csv")
if not valid_base_path.exists():
    valid_data["answers"] = get_answers(assistant, valid_data["questions"])
    valid_data.to_csv("valid_set_base.csv", index=False)
else:
    valid_data = pd.read_csv(valid_base_path)

#### Классические оценки

Далее сможем сравнивать с оценкой качества решения без каких-либо данных.

Метрики на голдах без модификаций:
1. BERTScore
2. BLEU
3. ROGUE

In [7]:
import evaluate

bertscore = evaluate.load("bertscore")
bleu = evaluate.load("bleu")

In [8]:
from rouge_score import rouge_scorer

rogue = rouge_scorer.RougeScorer(
    ['rouge1', 'rouge2', 'rougeL'], 
    use_stemmer=True
)

In [21]:
predictions = valid_data["answers"]
references = valid_data["gold_answers"]

In [22]:
bertscore.compute(predictions=predictions[:2], references=references[:2], lang="ru")

KeyboardInterrupt: 

Рассчитаем Rogue-N: Prec и Rec по совпадению N-грамм в эталонном тексте и нашем.
Rouge-L: Длина общей подпоследовательности из чисел

In [32]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(
    ['rouge1', 'rouge2', 'rougeL'], 
    use_stemmer=True
)
for i in range(len(valid_data)):
    print(scorer.score(target=references[i], prediction=predictions[i]))

{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
{'rouge1': Score(precision=0.2, recall=0.3333333333333333, fmeasure=0.25)}
{'rouge1': Score(precision=1.0, recall=0.3333333333333333, fmeasure=0.5)}
{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
{'rouge1': Score(precision=0.3333333333333333, recall=0.2, fmeasure=0.25)}
{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
{'rouge1': Score(precision=0.125, recall=0.3333333333333333, fmeasure=0.18181818181818182)}
{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
{'rouge1': Score(precision=0.0, rec

BLEU - допускает несколько правильных вариантов ответа.

In [43]:
bleu.compute(predictions=[references[0]], references=[references[0]])

{'bleu': 1.0,
 'precisions': [1.0, 1.0, 1.0, 1.0],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 11,
 'reference_length': 11}

In [9]:
def calc_classic_metrcis(data: pd.DataFrame):
    data_with_metrics = data.copy()
    
    answers = data_with_metrics["answers"]
    gold_answers = data_with_metrics["gold_answers"]
    
    print("calculating bert scores...")
    bert_scores = bertscore.compute(predictions=answers, references=gold_answers, lang="ru")
    data_with_metrics["bert_precision"] = bert_scores["precision"]
    data_with_metrics["bert_recall"] = bert_scores["recall"]
    data_with_metrics["bert_f1"] = bert_scores["f1"]
    
    print("calculating bleu scores...")
    bleu_scores = []
    for i in range(len(data_with_metrics)):
        item_bleu = bleu.compute(predictions=[answers[i]], references=[gold_answers[i]])["bleu"]
        bleu_scores.append(item_bleu)
    data_with_metrics["bleu"] = bleu_scores
    
    print("calculating rouge scores...")
    metric_names = ["precision", "recall", "fmeasure"]
    rouge_scores = {
        "rouge1": {metric_name: [] for metric_name in metric_names},
        "rouge2": {metric_name: [] for metric_name in metric_names},
        "rougeL": {metric_name: [] for metric_name in metric_names}
    }
    rouge = rouge_scorer.RougeScorer(
        list(rouge_scores.keys()), 
        use_stemmer=True
    )
    
    for i in range(len(data)):
        for key, value in rouge.score(target=answers[i], prediction=gold_answers[i]).items():
            for metric_name in metric_names:
                rouge_scores[key][metric_name].append(getattr(value, metric_name))
            
    for key in rouge_scores:
        data_with_metrics[key] = rouge_scores[key]
        for metric_name in metric_names:
            data_with_metrics[key + "_" + metric_name] = rouge_scores[key][metric_name]
            
    return data_with_metrics

In [10]:
calc_classic_metrcis(valid_data[:10]).describe()

calculating bert scores...
calculating bleu scores...
calculating rouge scores...


,bert_precision,bert_recall,bert_f1,bleu,rouge1_precision,rouge1_recall,rouge1_fmeasure,rouge2_precision,rouge2_recall,rouge2_fmeasure,rougeL_precision,rougeL_recall,rougeL_fmeasure
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.0,10.0,10.0,10.000000,10.000000,10.000000
mean,0.594179,0.702531,0.643082,0.005761,0.086667,0.153333,0.100000,0.0,0.0,0.0,0.086667,0.153333,0.100000
std,0.046573,0.040098,0.039126,0.018219,0.144188,0.319026,0.174801,0.0,0.0,0.0,0.144188,0.319026,0.174801
min,0.496684,0.650658,0.573065,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
25%,0.574072,0.673238,0.629999,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
50%,0.602383,0.704912,0.641272,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
75%,0.623815,0.719524,0.665404,0.000000,0.150000,0.150000,0.187500,0.0,0.0,0.0,0.150000,0.150000,0.187500
max,0.656844,0.787742,0.716363,0.057613,0.333333,1.000000,0.500000,0.0,0.0,0.0,0.333333,1.000000,0.500000


#### Оценка достоверности

1. Можно выделить сущности
2. Можно BLEU, ROGUE по округленным чиселками или по округленным чиселкам с ключевыми словами + юзерская

Посчитаем обычный numeric

In [20]:
from deeppavlov import build_model, configs

# Загружаем модель для NER
model = build_model(configs.ner.ner_ontonotes_ru, download=True)

# Пример текста на русском языке
text = ["Согласно последним данным, компания Яндекс приобрела стартап в Москве за 1 миллион долларов."]

# Обрабатываем текст
predictions = model(text)

# Выводим результат
for sentence in predictions:
    for word, tag in zip(sentence[0], sentence[1]):
        print(f"{word}: {tag}")


config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


Word: Сергей, Entity: U-FIRST_NAME, Score: 0.9982
Word: Иванович, Entity: U-MIDDLE_NAME, Score: 0.9976
Word: Москвы, Entity: U-CITY, Score: 0.9982


In [23]:
text = "12-12-2024 ты потратил 20 000 рублей"
results = ner_pipe(text)
print(results)

for result in results:
    print(f"Word: {result['word']}, Entity: {result['entity']}, Score: {result['score']:.4f}")

[]


In [12]:
import spacy

nlp = spacy.load("ru_ent_web_md")
doc = nlp("я заплатил $50 за книгу 2023-10-01.")
for ent in doc.ents:
   print(ent.text, ent.label_)

OSError: [E050] Can't find model 'ru_ent_web_md'. It doesn't seem to be a Python package or a valid path to a data directory.

In [34]:
def is_float(num: str):
    try:
        float(num)
        return True
    except ValueError as exc:
        return False

def score_numbers_accuracy(prediction, reference):
    prediction_numbers = " ".join([str(int(float(num))) for num in prediction.split() if is_float(num)])
    reference_numbers = " ".join([str(int(float(num))) for num in reference.split() if is_float(num)])
    
    rogue = rouge_scorer.RougeScorer(
        ['rouge1'], 
        use_stemmer=False
    )
    return rogue.score(reference_numbers, prediction_numbers)["rouge1"]

score_numbers_accuracy(valid_data["gold_answers"][0], valid_data["answers"][0])

KeyError: 'rogue1'

LLM:
Варианты: 
1. Судья оценивает ответ по своим критериям, но видит эталонные.
2. Судья по критериям оценивает схожесть ответа с эталонным (включая Accuracy)
3. Online-судья, который оценивает все, кроме достоверности. Оценка достоверности должна быть на юзере и кореллировать с автоматическими

Разметчики должны выставить нам оценки по его криетриям, чтобы мы замерили корреляцию


In [71]:
judge_prompt = """
Вы будете оценивать ответ финансового ассистента на вопрос пользователя, основываясь на следующих критериях. Для каждого критерия укажите свою оценку от 0 до 5 и предоставьте обоснование вашей оценки.

### Входные данные:
1. Вопрос пользователя: {question}
2. Ответ ассистента: {answer}
3. Верная информация: {gold_answer}

### Критерии оценки:

1. Правдивость:
   - Оцените, насколько ответ ассистента соответствует действительности и не содержит вымышленных или неточных данных. Например, если ассистент указывает сумму расходов, она должна быть в пределах реального контекста. Используйте сравнение с верной информацией, переданной во входных данных.

2. Помощь, полезность:
   - Оцените, насколько ответ ассистента полезен для пользователя и может помочь в решении его задачи. Например, все советы должны быть практичными и легко реализуемыми.

3. Калибровка:
   - Оцените, как ассистент демонстрирует неуверенность в тех случаях, когда правильный ответ неизвестен. Если ассистент содержит рекомендации по дополнительным ресурсам или действиям в условиях неопределенности, это положительно скажется на оценке.

4. Релевантность:
   - Оцените, насколько ответ соответствует заданному вопросу и его контексту. Все советы должны быть релевантными и отражать фактическую ситуацию пользователя.

5. Полнота:
    - Оцените, насколько всесторонне ответ охватывает вопрос. Если вопрос не подразумевает развернутого ответа, а ответ является лаконичным, оценка полноты должна быть высокой
"""

In [78]:
from dotenv import load_dotenv
load_dotenv()

from  pydantic import BaseModel
from typing import Annotated
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

class CriterionSchema(BaseModel):
    mark: Annotated[int, "Оценка ответа судьей - единственное число"]
    reasoning: Annotated[str, "Обоснование, по которому выставлена именно такая оценка"]
    
class JudgeSchema(BaseModel):
    accuracy: Annotated[CriterionSchema, "Точная оценка правдивости"]
    usefulness: Annotated[CriterionSchema, "Точная оценка полезности"]
    calibration: Annotated[CriterionSchema, "Точная оценка калибровки"]
    relevance: Annotated[CriterionSchema, "Точная оценка релевантности"]
    recall: Annotated[CriterionSchema, "Точная оценка полноты"]
    
llm = ChatPromptTemplate.from_template(judge_prompt) | ChatOpenAI(model="gpt-4o-mini").with_structured_output(JudgeSchema)

In [79]:
# for i in range(2):
#     question = valid_data["questions"][i]
#     answer = valid_data["answers"][i]
#     gold_answer = valid_data["gold_answers"][i]
# 
#     print(llm.invoke({
#         "question": question,
#         "answer": answer,
#         "gold_answer": gold_answer
#     }))

accuracy=CriterionSchema(mark=2, reasoning='Ответ ассистента не содержит точной информации о дате платежа, что является ключевым аспектом вопроса пользователя. Ассистент признает, что у него нет точной информации, что снижает точность ответа.') usefulness=CriterionSchema(mark=4, reasoning='Ответ содержит полезные рекомендации о том, как узнать информацию о платеже, что может помочь пользователю в решении его задачи.') calibration=CriterionSchema(mark=5, reasoning='Ассистент адекватно демонстрирует неуверенность, указывая на отсутствие точной информации и предлагая альтернативные способы получения необходимых данных.') relevance=CriterionSchema(mark=4, reasoning='Ответ в целом соответствует вопросу, предлагая советы, которые могут помочь пользователю, хотя и не дает конкретного ответа на заданный вопрос.') recall=CriterionSchema(mark=1, reasoning='Ассистент не сумел предоставить ответ на вопрос о конкретной дате платежа, что является ключевым моментом.')
accuracy=CriterionSchema(mark=2,

1. NER метрика
2. Судья - все, кроме точности Accuracy
3. Визализации и новы йрпогон

In [11]:
from dotenv import load_dotenv
load_dotenv()

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch
import os

# Загрузите модель DeepPavlov для NER из Hugging Face
model_name = "DeepPavlov/rubert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Создание пайплайна для NER
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In [28]:
import spacy

ner = spacy.load("en_core_web_lg")  # или "en_core_web_lg" или "en_core_web_trf"
print(ner.get_pipe("ner").labels)

text = "Apple is looking at buying U.K. startup for $1 billion at 23/03/2024"
doc = ner(text)

# Выводим найденные сущности


('CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART')


In [30]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Apple ORG
U.K. GPE
$1 billion MONEY
23/03/2024 DATE


In [46]:
from deep_translator import GoogleTranslator
from rouge_score import rouge_scorer

def ner_accuracy(gold_answer: str, answer: str):
    translator = GoogleTranslator(source="ru", target="en")
    
    gold_answer = translator.translate(gold_answer)
    answer = translator.translate(answer)
    
    target_ents = ["DATE", "EVENT", "MONEY", "TIME"]
    
    gold_answer_by_ents = {ent_name: [] for ent_name in target_ents}
    answer_by_ents = {ent_name: [] for ent_name in target_ents}
    
    for ent in ner(gold_answer).ents:
        if ent.label_ in target_ents:
            gold_answer_by_ents[ent.label_].append(ent.text)
    
    for ent in ner(answer).ents:
        if ent.label_ in target_ents:
            answer_by_ents[ent.label_].append(ent.text)
                
    rouge = rouge_scorer.RougeScorer(["rouge1"], use_stemmer=False)
    
    gold_answer_by_ents = {key: " ".join(list(set(value))) for key, value in gold_answer_by_ents.items()}
    answer_by_ents = {key: " ".join(list(set(value))) for key, value in answer_by_ents.items()}
    
    precision = 0
    recall = 0
    f1 = 0
    for ent in target_ents:   
        res = rouge.score(target=gold_answer_by_ents[ent], prediction=answer_by_ents[ent])["rouge1"]
        
        precision += res.precision
        recall += res.recall
        f1 += res.fmeasure
        
    print(precision / len(target_ents))
    print(recall / len(target_ents))
    print(f1 / len(target_ents))
    
valid_data = pd.read_csv("valid_set.csv")
ner_accuracy(valid_data["gold_answers"][0], valid_data["answers"][0])           

{'DATE': [], 'EVENT': [], 'MONEY': ['9420 rubles'], 'TIME': []} {'DATE': ['2024', '2024'], 'EVENT': [], 'MONEY': [], 'TIME': []}
0.0
0.0
0.0


In [48]:
def calc_ner_accuracy(data: pd.DataFrame):
    ner_acc = []
    for i in range(len(data)):
        ner_acc.append(ner_accuracy(data["gold_answers"][i], data["answers"][i]))
    data_with_metric = data.copy()
    data_with_metric["ner_acc"] = ner_acc
    return data_with_metric

calc_ner_accuracy(valid_data)

{'DATE': [], 'EVENT': [], 'MONEY': ['9420 rubles'], 'TIME': []} {'DATE': ['2024', '2024'], 'EVENT': [], 'MONEY': [], 'TIME': []}
0.0
0.0
0.0
{'DATE': ['2024'], 'EVENT': [], 'MONEY': ['2664 rubles'], 'TIME': []} {'DATE': ['2024'], 'EVENT': [], 'MONEY': ['2664 rubles'], 'TIME': []}
0.5
0.5
0.5
{'DATE': [], 'EVENT': [], 'MONEY': [], 'TIME': []} {'DATE': ['December 8, 2024'], 'EVENT': [], 'MONEY': [], 'TIME': []}
0.0
0.0
0.0
{'DATE': [], 'EVENT': [], 'MONEY': [], 'TIME': []} {'DATE': ['2024-12-10'], 'EVENT': [], 'MONEY': ['12345'], 'TIME': []}
0.0
0.0
0.0
{'DATE': [], 'EVENT': [], 'MONEY': ['290 rubles'], 'TIME': []} {'DATE': ['June 3, 2024'], 'EVENT': [], 'MONEY': ['290 rubles'], 'TIME': []}
0.25
0.25
0.25
{'DATE': [], 'EVENT': [], 'MONEY': ['10,000 rubles'], 'TIME': []} {'DATE': ['December 8, 2024'], 'EVENT': [], 'MONEY': [], 'TIME': []}
0.0
0.0
0.0
{'DATE': ['October 29'], 'EVENT': [], 'MONEY': [], 'TIME': []} {'DATE': ['December 4, 2024', '11:23:29'], 'EVENT': [], 'MONEY': [], 'TIME': 

KeyboardInterrupt: 

In [15]:
nlp.get_pipe("ner").labels

('LOC', 'ORG', 'PER')